# General Usage

In [1]:
from pprint import pprint
import tdameritrade as td
import os

## Authentication



In [2]:
client_id = os.getenv('TDAMERITRADE_CLIENT_ID')
account_id = os.getenv('TDAMERITRADE_ACCOUNT_ID')
refresh_token = os.getenv('TDAMERITRADE_REFRESH_TOKEN')
redirect_uri = os.getenv('TDAMERITRADE_REDIRECT_URI')

#Uncomment the 3 lines below for debugging purposes
#pprint(client_id)
#pprint(account_id)
#pprint(refresh_token)

In [3]:
from tdameritrade import auth

tdauser=os.getenv('TDA_USER')
tdapass=os.getenv('TDA_PASS')

#Uncomment the 4 lines below for debugging purposes
#pprint(client_id)
#pprint(redirect_uri)
#pprint(tdauser)
#pprint(tdapass)

# authenticate client using ThinkorSwim username and password; store refresh token
tokens = td.auth.authentication(client_id=client_id, redirect_uri=redirect_uri, tdauser=tdauser, tdapass=tdapass)
#pprint(tokens) #Uncomment this line for debugging




In [4]:
# initialize TDClient using refresh_token
td_client = td.TDClient(client_id=client_id, refresh_token=tokens['refresh_token'])

# store refresh_token in a hidden file called .refresh_token in home directory
refreshtoken = open(os.path.expanduser('~/.r_token'), 'w')
print(tokens['refresh_token'], file = refreshtoken, end = "")
refreshtoken.close()


## API Examples

In [5]:
# shows private account info
# pprint(td_client.accounts())

### Getting Symbol Data

In [6]:
pprint(td_client.search('aapl'))

{'AAPL': {'assetType': 'EQUITY',
          'cusip': '037833100',
          'description': 'Apple Inc. - Common Stock',
          'exchange': 'NASDAQ',
          'symbol': 'AAPL'}}


In [7]:
pprint(td_client.quote('aapl'))

{'AAPL': {'52WkHigh': 145.09,
          '52WkLow': 53.1525,
          'askId': 'K',
          'askPrice': 136.43,
          'askSize': 100,
          'assetMainType': 'EQUITY',
          'assetType': 'EQUITY',
          'bidId': 'K',
          'bidPrice': 136.4,
          'bidSize': 100,
          'bidTick': ' ',
          'closePrice': 133.94,
          'cusip': '037833100',
          'delayed': False,
          'description': 'Apple Inc. - Common Stock',
          'digits': 4,
          'divAmount': 0.82,
          'divDate': '2021-02-05 00:00:00.000',
          'divYield': 0.61,
          'exchange': 'q',
          'exchangeName': 'NASD',
          'highPrice': 0.0,
          'lastId': 'P',
          'lastPrice': 136.4,
          'lastSize': 0,
          'lowPrice': 0.0,
          'marginable': True,
          'mark': 136.4,
          'markChangeInDouble': 2.46,
          'markPercentChangeInDouble': 1.8366,
          'nAV': 0.0,
          'netChange': 2.46,
          'netPercentCha

In [8]:
pprint(td_client.instrument('037833100'))

[{'assetType': 'EQUITY',
  'cusip': '037833100',
  'description': 'Apple Inc. - Common Stock',
  'exchange': 'NASDAQ',
  'symbol': 'AAPL'}]


In [9]:
import pandas as pd
pd.DataFrame(td_client.history('AAPL')['candles']).head()

,open,high,low,close,volume,datetime
0,133.68,133.75,133.57,133.75,16808,1611230400000
1,133.73,133.83,133.73,133.80,26322,1611230460000
2,133.80,133.86,133.76,133.82,13261,1611230520000
3,133.85,133.98,133.85,133.95,27051,1611230580000
4,133.95,133.98,133.90,133.90,14793,1611230640000


### Getting Option Data

In [10]:
options = td_client.options('AAPL')

In [11]:
expiry_dates = list(options['putExpDateMap'].keys())
expiry = expiry_dates[0]
print(f"Expiry Date: {expiry}")

strike_prices = list(options['putExpDateMap'][expiry].keys())
strike = strike_prices[0]
print(f"Strike Price: {strike}")

pprint(options['putExpDateMap'][expiry][strike])

Expiry Date: 2021-02-05:1
Strike Price: 65.0
[{'ask': 0.01,
  'askSize': 420,
  'bid': 0.0,
  'bidAskSize': '0X420',
  'bidSize': 0,
  'closePrice': 0.0,
  'daysToExpiration': 1,
  'deliverableNote': '',
  'delta': 0.0,
  'description': 'AAPL Feb 5 2021 65 Put (Weekly)',
  'exchangeName': 'OPR',
  'expirationDate': 1612558800000,
  'expirationType': 'S',
  'gamma': 0.0,
  'highPrice': 0.0,
  'inTheMoney': False,
  'isIndexOption': None,
  'last': 0.01,
  'lastSize': 0,
  'lastTradingDay': 1612573200000,
  'lowPrice': 0.0,
  'mark': 0.01,
  'markChange': 0.0,
  'markPercentChange': 4900.0,
  'mini': False,
  'multiplier': 100.0,
  'netChange': 0.01,
  'nonStandard': False,
  'openInterest': 2510,
  'openPrice': 0.0,
  'optionDeliverablesList': None,
  'percentChange': 9900.0,
  'putCall': 'PUT',
  'quoteTimeInLong': 1612385999605,
  'rho': 0.0,
  'settlementType': ' ',
  'strikePrice': 65.0,
  'symbol': 'AAPL_020521P65',
  'theoreticalOptionValue': 0.0,
  'theoreticalVolatility': 29.0,


## Placing an Order

### Creating order body for Place Order API

In [12]:
from tdameritrade.orders.order_builder import build_buy_market_stock_order

# Prints function docstring
# build_buy_market_stock_order?

In [13]:
symbol = "HUSA"  # Cheap stock for example
quantity = 1
order = build_buy_market_stock_order(symbol, quantity)

In [14]:
pprint(order.asdict())

{'duration': 'DAY',
 'orderLegCollection': [{'instruction': 'BUY',
                         'instrument': {'assetType': 'EQUITY',
                                        'symbol': 'HUSA'},
                         'quantity': 1}],
 'orderStrategyType': 'SINGLE',
 'orderType': 'MARKET',
 'session': 'NORMAL'}


### Submitting Order

In [15]:
# Without this, it fails with a 415.
# This may screw up other API calls, so it may have to be reset if you want to try other ones.
td_client.session.headers = {
    "Content-Type": "application/json"
}

In [16]:
# This will fail to return because the place order API does not return a JSONserializable response on success
# BUT the order will still go through.  Checked on the site.  You can also check your active orders below.

# try:
#     response = td_client.placeOrder(account_id, order.json())
#     pprint(response)
# except json.decoder.JSONDecodeError as err:
#     print(err)


### Getting out Orders

In [17]:
# Load account info to get account ID for get orders
td_client.accounts()

account_id = td_client.accountIds[0]

In [18]:
# Print out orders
my_orders = td_client.orders(accountId=account_id)
pprint(my_orders)

[]


### Deleting Orders

In [19]:
# This deletes all WORKING orders

# for order in my_orders:
#     if order["status"] == Status.WORKING and order["cancelable"]:
#         response = td_client.cancelOrder(account_id, order["orderId"])
#         pprint(response)